In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import shap
from tensorflow.python.framework.ops import disable_eager_execution

In [2]:
import math
from statsmodels.tsa.stattools import adfuller

In [3]:
import statsmodels.api as sm
from sklearn.decomposition import PCA

In [4]:
df = pd.read_excel('newrbi.xlsx')

In [ ]:
df

In [5]:
df.drop(['Months'],axis=1)

,Coal,Crude Oil,Natural Gas,Petroleum Refinery Products,Fertilizers,Steel,Cement,Electricity Generation,Domestic Gold,CPI,WPI,BSE,NSE,Exports,Imports,IIP
0,43.114,3142.679,3547.627680,16654.998186,2307.4,6695,20430,75261.21,28562.730000,100.000000,104.7,17318.81,5254.480000,104134,162376,99.3
1,45.691,3249.106,3606.596136,17637.881393,2716.3,7513,20491,79432.56,28830.960000,100.000000,105.3,16218.53,4966.506818,119095,203212,105.0
2,42.623,3140.799,3460.417392,17603.791175,3016.7,6579,19740,76803.26,29818.620000,100.000000,105.3,17429.98,5074.214286,119024,183225,102.3
3,40.353,3255.294,3507.717776,18068.521623,3365.5,7083,19430,76470.66,29468.460000,100.000000,106.2,17236.18,5222.011364,117380,182582,101.5
4,36.951,3228.661,3474.253850,18121.076406,3389.9,6442,17740,74498.36,30179.790000,100.000000,106.9,17429.56,5329.721429,112148,181046,99.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,45.146,2577.486,2363.475000,17936.831000,3902.5,9504,20871,118898.91,52917.190000,154.700000,122.0,38628.29,11318.570000,184893,283530,117.2
101,48.530,2486.523,2227.733000,18000.818000,3669.4,9454,24244,121584.95,50783.640000,156.450000,122.9,38067.93,11030.484435,185566,268891,124.1
102,56.268,2567.704,2347.808000,18876.540000,3968.5,9989,27030,118515.59,50697.850000,158.400000,123.6,39614.07,11095.728890,186211,269853,129.2
103,63.400,2486.012,2262.971000,21425.494000,3829.2,9721,25256,105853.22,50428.650000,158.882957,125.1,44149.72,11160.190492,183949,275255,126.1


In [ ]:
X = df.drop(['IIP','Months/Years'],axis=1)
y = df['IIP']
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,shuffle=False)

In [ ]:
normalizer = preprocessing.Normalizer().fit(X_train)
X_train_norm = normalizer.fit_transform(X_train)
X_test_norm = normalizer.fit_transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(25, input_shape=(13,),activation = 'relu'))
model.add(Dense(20,activation = 'relu'))
model.add(Dense(1,activation = 'relu'))
model.compile(optimizer = SGD(lr = 0.001), loss = 'mean_squared_error', metrics = ['mean_squared_error'])

In [ ]:
network = model.fit(X_train_norm, y_train, epochs = 5000, batch_size = 53, verbose = 1)

In [ ]:
prediction = model.predict(X_test_norm, verbose = 1)

In [ ]:
prediction

In [ ]:
y_test

In [ ]:
print(mean_squared_error(y_test,prediction))

In [ ]:
for columns in df.columns:
    result = adfuller(np.log(df[columns]), autolag='AIC')

In [ ]:
result = adfuller(np.log(df['IIP']),autolag='AIC')

In [ ]:
result

In [ ]:
result = adfuller(np.log(df['IIP']),autolag='AIC')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
 print('\t%s: %.3f' % (key, value))

In [6]:
df_new = pd.read_excel('latest.xlsx',sheet_name='Sheet4')

In [7]:
df_new

,Months,Coal,Crude Oil,Natural Gas,Petroleum Refinery Products,Fertilizers,Steel,Cement,Electricity Generation,Domestic Gold,CPI,WPI,BSE,NSE,Exports,Imports,IIP
0,2012-04-01,0.058054,0.033304,0.016485,0.057339,0.163149,0.115274,0.002981,0.053943,0.009347,0.000000,0.005714,-0.065639,-0.056364,0.134243,0.224335,0.055815
1,2012-05-01,-0.069507,-0.033903,-0.041375,-0.001935,0.104893,-0.132752,-0.037339,-0.033661,0.033683,0.000000,0.000000,0.072037,0.021455,-0.000596,-0.103535,-0.026051
2,2012-06-01,-0.054728,0.035805,0.013576,0.026057,0.109413,0.073815,-0.015829,-0.004340,-0.011812,0.000000,0.008511,-0.011181,0.028711,-0.013909,-0.003516,-0.007851
3,2012-07-01,-0.088073,-0.008215,-0.009586,0.002904,0.007224,-0.094858,-0.090996,-0.026130,0.023852,0.000000,0.006570,0.011157,0.020416,-0.045597,-0.008448,-0.015889
4,2012-08-01,0.004078,-0.052728,-0.061096,-0.059186,0.011206,-0.030258,0.055905,-0.016641,0.048287,0.000000,0.006527,0.073705,0.028766,0.120581,0.044962,-0.009050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2020-07-20,-0.020911,-0.021533,-0.002543,-0.077744,0.011157,0.059724,-0.149932,-0.021985,0.059140,0.004211,0.008230,0.026798,0.037893,0.020231,0.035724,-0.005955
100,2020-08-20,0.072280,-0.035929,-0.059149,0.003561,-0.061589,-0.005275,0.149809,0.022340,-0.041154,0.011249,0.007350,-0.014613,-0.025782,0.003633,-0.053012,0.057206
101,2020-09-20,0.147944,0.032127,0.052498,0.047503,0.078360,0.055047,0.108778,-0.025569,-0.001691,0.012387,0.005680,0.039812,0.005897,0.003470,0.003571,0.040274
102,2020-10-20,0.119338,-0.032332,-0.036804,0.126662,-0.035732,-0.027196,-0.067884,-0.112991,-0.005324,0.003044,0.012063,0.108402,0.005793,-0.012222,0.019821,-0.024286


In [8]:
df_cycle = pd.DataFrame()
for columns in df_new.columns[1:]:
    cycle, trend = sm.tsa.filters.hpfilter(df_new[columns], 129600)
    df_cycle[columns] = cycle
    

In [9]:
df_cycle

,Coal,Crude Oil,Natural Gas,Petroleum Refinery Products,Fertilizers,Steel,Cement,Electricity Generation,Domestic Gold,CPI,WPI,BSE,NSE,Exports,Imports,IIP
0,0.051388,0.032736,0.028676,0.050627,0.140234,0.110841,0.000456,0.048222,0.009758,-0.005958,0.002340,-0.077521,-0.067511,0.116095,0.205948,0.051083
1,-0.076064,-0.034415,-0.029453,-0.008475,0.082774,-0.137144,-0.039899,-0.039421,0.034165,-0.005927,-0.003287,0.060230,0.010314,-0.018261,-0.121332,-0.030712
2,-0.061177,0.035348,0.025230,0.019688,0.088089,0.069463,-0.018424,-0.010138,-0.011260,-0.005896,0.005311,-0.022912,0.017577,-0.031091,-0.020724,-0.012442
3,-0.094413,-0.008617,0.001799,-0.003294,-0.013308,-0.099171,-0.093627,-0.031967,0.024474,-0.005864,0.003457,-0.000497,0.009289,-0.062298,-0.025070,-0.020411
4,-0.002153,-0.053076,-0.049980,-0.065216,-0.008539,-0.034531,0.053241,-0.022517,0.048978,-0.005833,0.003501,0.062130,0.017648,0.104359,0.028923,-0.013503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,-0.033661,-0.017685,0.000489,-0.079153,0.007694,0.047797,-0.156956,-0.022430,0.039606,-0.000962,0.006056,0.011543,0.036721,0.015403,0.032836,-0.011946
100,0.058980,-0.032075,-0.056081,0.002048,-0.065188,-0.017744,0.142428,0.021960,-0.061058,0.006009,0.005147,-0.030282,-0.026816,-0.001220,-0.055788,0.050904
101,0.134086,0.035986,0.055600,0.045883,0.074626,0.042034,0.101038,-0.025882,-0.021965,0.007080,0.003449,0.023725,0.005001,-0.001410,0.000906,0.033660
102,0.104921,-0.028468,-0.033667,0.124935,-0.039600,-0.040753,-0.075984,-0.113238,-0.025968,-0.002330,0.009804,0.091895,0.005034,-0.017128,0.017265,-0.031213


In [15]:
pca = PCA(n_components=1)
pca.fit(df_cycle.drop(['IIP'],axis=1))


PCA(n_components=1)

In [16]:
pca.components_[0]

array([-2.86035848e-01, -5.15471414e-02, -7.14173484e-02, -1.39372994e-01,
       -1.25965120e-01, -5.95705266e-01, -7.01150239e-01, -1.00966108e-01,
        1.91044397e-02,  5.25626479e-03,  5.73953473e-04,  2.53417570e-02,
       -1.76135363e-02, -1.15980290e-01, -6.05518135e-02])

In [17]:
df_cycle.columns

Index(['Coal ', ' Crude Oil ', 'Natural Gas ', 'Petroleum Refinery Products',
       ' Fertilizers ', 'Steel ', 'Cement ', ' Electricity Generation ',
       'Domestic Gold', 'CPI', 'WPI ', 'BSE ', 'NSE', 'Exports', ' Imports',
       'IIP'],
      dtype='object')

In [19]:
df_comp = pd.DataFrame()
for columns in df_cycle.columns[:-1]:
    print(columns,pca.components_[0][df_cycle.columns.get_loc(columns)])
    df_comp[columns] = (pca.components_[0][df_cycle.columns.get_loc(columns)])*(df_cycle[columns])

Coal  -0.2860358484285419
 Crude Oil  -0.05154714142870298
Natural Gas  -0.07141734835474714
Petroleum Refinery Products -0.13937299412168724
 Fertilizers  -0.12596512003064786
Steel  -0.5957052663611198
Cement  -0.7011502387500881
 Electricity Generation  -0.10096610825011104
Domestic Gold 0.019104439715513337
CPI 0.005256264789938318
WPI  0.0005739534734102677
BSE  0.02534175701640199
NSE -0.017613536303607978
Exports -0.11598028951571501
 Imports -0.06055181346350366


In [20]:
df_comp

,Coal,Crude Oil,Natural Gas,Petroleum Refinery Products,Fertilizers,Steel,Cement,Electricity Generation,Domestic Gold,CPI,WPI,BSE,NSE,Exports,Imports
0,-0.014699,-0.001687,-0.002048,-0.007056,-0.017665,-0.066029,-0.000320,-0.004869,0.000186,-0.000031,0.000001,-0.001965,0.001189,-0.013465,-0.012471
1,0.021757,0.001774,0.002103,0.001181,-0.010427,0.081697,0.027975,0.003980,0.000653,-0.000031,-0.000002,0.001526,-0.000182,0.002118,0.007347
2,0.017499,-0.001822,-0.001802,-0.002744,-0.011096,-0.041380,0.012918,0.001024,-0.000215,-0.000031,0.000003,-0.000581,-0.000310,0.003606,0.001255
3,0.027006,0.000444,-0.000128,0.000459,0.001676,0.059076,0.065646,0.003228,0.000468,-0.000031,0.000002,-0.000013,-0.000164,0.007225,0.001518
4,0.000616,0.002736,0.003569,0.009089,0.001076,0.020570,-0.037330,0.002273,0.000936,-0.000031,0.000002,0.001574,-0.000311,-0.012104,-0.001751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.009628,0.000912,-0.000035,0.011032,-0.000969,-0.028473,0.110050,0.002265,0.000757,-0.000005,0.000003,0.000293,-0.000647,-0.001786,-0.001988
100,-0.016870,0.001653,0.004005,-0.000285,0.008211,0.010570,-0.099863,-0.002217,-0.001166,0.000032,0.000003,-0.000767,0.000472,0.000142,0.003378
101,-0.038354,-0.001855,-0.003971,-0.006395,-0.009400,-0.025040,-0.070843,0.002613,-0.000420,0.000037,0.000002,0.000601,-0.000088,0.000164,-0.000055
102,-0.030011,0.001467,0.002404,-0.017413,0.004988,0.024277,0.053276,0.011433,-0.000496,-0.000012,0.000006,0.002329,-0.000089,0.001987,-0.001045


In [21]:
df_comp['Index'] = df_comp.sum(axis=1)

In [22]:
df_comp['Index']

0     -0.140926
1      0.141471
2     -0.023676
3      0.166413
4     -0.009084
         ...   
99     0.101035
100   -0.092704
101   -0.153003
102    0.053101
103   -0.169072
Name: Index, Length: 104, dtype: float64

In [25]:
print(mean_squared_error(df_cycle['IIP'],df_comp['Index']))

0.20314865581033015
